In [2]:
from langchain import PromptTemplate, LLMChain
from langchain_community.chat_models.perplexity import ChatPerplexity
from langchain.chains import SequentialChain

In [3]:
from dotenv import load_dotenv
import os
dotenv_file = '/home/amir/source/.env'
load_dotenv(dotenv_file)
PPLX_API_KEY = os.getenv("PPLX_API_KEY")


In [4]:
# --------------- Iniitialization ----------------
# Initialize the language model
llm = ChatPerplexity(model="sonar-reasoning",
                     temperature=0.5, # lower is more deterministic
                     pplx_api_key=PPLX_API_KEY)

In [5]:
# ---------------- Chain definition ----------------
# 1. Market data analyst: retrieve the latest market data
market_data_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ticker"],
        template=(
            "✅ For {ticker}, provide detailed market data including the latest price, volume, key financials ratios "
            "(e.g., P/E, P/B, P/S, dividient yield), recent price trends, and relevant market indications."
        ),
    ),
    output_key="market_data",
)

# 2. Sentiment Analyst: Analyze news and social media sentiment
sentiment_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ticker"],
        template=(
            "📰 For {ticker}, analyze recent news articles, social media posts, and expert commentary. "
            "Summarize the prevailing sentiment, highlight any key events, and note emerging trends that may impact the stock."
        )
    ),
    output_key="sentiment_analysis"
)

# 3. Macro-Economic Analyst: Evaluate macro-economic conditions
macro_analysis_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["ticker"],
        template=(
            "🌐 For {ticker}, analyze the current macro-economic environment. "
            "Include key indicators such as GDP growth, inflation rates, interest rates, unemployment trends, "
            "and central bank policies. Summarize how these factors could impact the overall market and the asset."
        )
    ),
    output_key="macro_analysis"
)

# 4. Quantitative Strategist: Develop a trading strategy
strategy_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["market_data", "sentiment_analysis", "macro_analysis"],
        template=(
            "📊 Using the detailed market data:\n{market_data}\n"
            "the sentiment analysis:\n{sentiment_analysis}\n"
            "and the macro-economic analysis:\n{macro_analysis}\n"
            "develop a sophisticated trading strategy. Outline a clear asset allocation, specify entry and exit points, "
            "detail risk management measures, and provide estimated expected returns. If applicable, incorporate algorithmic signals."
        )
    ),
    output_key="strategy"
)

# 5. Risk Manager: Assess the strategy's risk
risk_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["strategy"],
        template=(
            "⚠️ Evaluate the following trading strategy:\n{strategy}\n"
            "Identify potential risks such as market volatility, liquidity issues, or unexpected market events. "
            "Summarize your risk assessment in 4 concise bullet points, and state in the final bullet point whether the strategy meets an acceptable risk tolerance."
        )
    ),
    output_key="risk_assessment"
)


/tmp/ipykernel_253162/3571652375.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  market_data_chain = LLMChain(


In [6]:
# -------------------- Sequential Orchestration --------------------
sequential_agent = SequentialChain(
    chains=[market_data_chain, sentiment_chain, macro_analysis_chain, strategy_chain, risk_chain],
    input_variables=["ticker"],
    output_variables=["market_data", "sentiment_analysis", "macro_analysis", "strategy", "risk_assessment"],
    verbose=True
)

In [7]:
# -------------------- Run the Analysis --------------------
def run_ai_hedge_fund(ticker: str) -> None:
    result = sequential_agent({"ticker": ticker})
    
    print("\n======== AI Hedge Fund Analysis Results ========\n")
    
    print("📈 Market Data Retrieved:")
    print("--------------------------------------------------")
    print(result["market_data"], "\n")
    
    print("📰 Market Sentiment Analysis:")
    print("--------------------------------------------------")
    print(result["sentiment_analysis"], "\n")
    
    print("🌐 Macro-Economic Analysis:")
    print("--------------------------------------------------")
    print(result["macro_analysis"], "\n")
    
    print("📊 Developed Trading Strategy:")
    print("--------------------------------------------------")
    print(result["strategy"], "\n")
    
    print("⚠️ Risk Assessment:")
    print("--------------------------------------------------")
    print(result["risk_assessment"], "\n")
    
    print("==============================================\n")

In [8]:
# Run the analysis for a given ticker (e.g., "MSFT")
run_ai_hedge_fund("NVDA")


/tmp/ipykernel_253162/2701550132.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = sequential_agent({"ticker": ticker})




> Entering new SequentialChain chain...

> Finished chain.

======== AI Hedge Fund Analysis Results ========

📈 Market Data Retrieved:
--------------------------------------------------
<think>
Okay, let's tackle this query about NVDA. The user wants detailed market data, so I need to gather the latest price, volume, financial ratios, price trends, and market indications from the search results provided.

First, looking at the search results. Result [1] is a YouTube video from Wicked Stocks discussing levels to watch for March 6th, 2025. They mention critical support and resistance levels, like 107.23 and 136.73. They also talk about potential price movements based on closing above or below certain levels. This seems useful for recent price trends and market indications.

Result [2] from Moomoo gives key financial indicators. The latest price here is $116.798, up 2.40%. Volume is 262.49M shares. P/E (TTM) is 39.73, P/B is 35.93, and dividend yield is 0.03%. These are crucial financia